# Исследование надёжности заёмщиков

Заказчик: кредитный отдел банка. 

Входные данные от банка: статистика о платёжеспособности клиентов.

Задача: Оценить, влияет ли семейное положение, количество детей, цели кредита и уровень дохода клиента на факт погашения кредита в срок. 

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<h2 id="tocheading">Содержание исследования</h2>


1. [Изучение общей информации о датасете, выявление некорректных данных.](#point1)
  * [Вывод](#conclusion1)
2. [Предобработка данных:](#point2)
  * [Обработка аномалий](#subpoint2_1)
  * [Обработка пропусков](#subpoint2_2)
  * [Замена типа данных](#subpoint2_3)
  * [Обработка дубликатов](#subpoint2_4)
  * [Лемматизация](#subpoint2_5)
  * [Категоризация данных](#subpoint2_6)
3. [Расчеты для ответа на поставленные в задаче вопросы.](#point3)
4. [Ответы на поставленные в задаче вопросы](#point4)
  * [Есть ли зависимость между наличием детей и возвратом кредита в срок?](#question1)
  * [Есть ли зависимость между семейным положением и возвратом кредита в срок?](#question2)
  * [Есть ли зависимость между уровнем дохода и возвратом кредита в срок?](#question3)
  * [есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?](#question4)
5. [Общий вывод](#point5)


### 1. Изучение общей информации о датасете, выявление некорректных данных. <a class="anchor" id="point1"></a>

Импортируем библиотеки

In [1]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

Посмотрим общую информацию о датафрейме

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Посмотрим наименования столбцов

In [5]:
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Изучим начало датафрейма

In [6]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Изучим конец датафрейма

In [7]:
df.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


Рассмотрим уникальные значения, имеющиеся в каждом столюбце, выявим наличие некорректных значений

In [8]:
columns = ['children', 'dob_years', 'education', 'education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'purpose']
for column in columns:
    print(column, df[column].sort_values().unique())

children [-1  0  1  2  3  4  5 20]
dob_years [ 0 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41
 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65
 66 67 68 69 70 71 72 73 74 75]
education ['ВЫСШЕЕ' 'Высшее' 'НАЧАЛЬНОЕ' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Начальное'
 'Неоконченное высшее' 'СРЕДНЕЕ' 'Среднее' 'УЧЕНАЯ СТЕПЕНЬ'
 'Ученая степень' 'высшее' 'начальное' 'неоконченное высшее' 'среднее'
 'ученая степень']
education_id [0 1 2 3 4]
family_status ['Не женат / не замужем' 'в разводе' 'вдовец / вдова' 'гражданский брак'
 'женат / замужем']
family_status_id [0 1 2 3 4]
gender ['F' 'M' 'XNA']
income_type ['безработный' 'в декрете' 'госслужащий' 'компаньон' 'пенсионер'
 'предприниматель' 'сотрудник' 'студент']
debt [0 1]
purpose ['автомобили' 'автомобиль' 'высшее образование'
 'дополнительное образование' 'жилье' 'заняться высшим образованием'
 'заняться образованием' 'на покупку автомобиля'
 'на покупку подержанного автомобиля' 'на покупку своего автомоб

Рассмотрим значения в столбце трудового стажа в днях, переведем в годы

In [9]:
print((df['days_employed']/365).dropna().sort_values().head(10))
print((df['days_employed']/365).dropna().sort_values().tail(10))

16335   -50.380685
4299    -48.261817
7329    -45.461569
17838   -44.560821
16825   -44.163528
3974    -43.385550
1539    -43.248435
4321    -43.213867
7731    -42.789216
15675   -42.219290
Name: days_employed, dtype: float64
8369     1100.247814
10991    1100.251585
17823    1100.313632
13420    1100.327762
4697     1100.369953
7794     1100.448904
2156     1100.477991
7664     1100.479708
10006    1100.591265
6954     1100.699727
Name: days_employed, dtype: float64


Рассмотрим значения в столбце месячного дохода

In [10]:
print((df['total_income']/1000).dropna().sort_values().head(10))
print((df['total_income']/1000).dropna().sort_values().tail(10))
print((df['total_income']/1000).dropna().sort_values().median())
print((df['total_income']/1000).dropna().sort_values().mean())

14585    20.667264
13006    21.205281
16174    21.367648
1598     21.695102
14276    21.895614
10881    22.472755
18509    23.844706
9070     24.457667
10068    25.227894
12052    25.308587
Name: total_income, dtype: float64
11071    1286.280999
15268    1350.245605
18353    1427.934463
18368    1551.152894
17503    1597.613490
17178    1711.309268
20809    1715.018393
9169     1726.276014
19606    2200.852210
12412    2265.604029
Name: total_income, dtype: float64
145.01793753253992
167.42230220817297


#### Вывод <a class="anchor" id="conclusion1"></a>

В таблице 12 столбцов, 21525 строк. Имена столбцов в одинаковом регистре (строчные буквы), не содержат пробелов, но не все отражают суть хранящейся в них информации, лучше переименовать. В двух столбцах - 'days_employed', 'total_income' - имеются пропущенные значения, которые необходимо рассмотреть и обработать. Тип данных в столбцах корректный в целом корректный, но трудовой стаж 'days_employed' и месячный доход 'total_income' проще было бы воспринимать в целочисленном виде. Из первых строк таблицы видно, что необходима предобработка данных в столбцах:
- 'days_employed' — содержит отрицательные значения, а также очень большие нереальные значения - если разделить на 365, то получим 1000 лет стажа; 
- 'education' — одинаковые значения указаны в разных регистрах; 
- 'purpose' — одинаковые цели кредита описаны разными словосочетаниями.
- 'children' - есть отрицательно значение в количестве детей "-1", а также значение 20 детей у нескольких клиентов (совпадение?!) - слишком много.
- 'dob_years' - содержатся нулевые значения в возрасте клиента.
- 'gender' - наличие некорректных значений пола клиента - 'XNA'.
- 'total_income' - есть слишком большие значения.

### 2. Предобработка данных <a class="anchor" id="point2"></a>

Переименуем столбцы

In [11]:
columns = ['children', 
           'days_employed', 
           'age', 
           'education', 
           'education_id', 
           'family_status', 
           'family_status_id', 
           'gender', 
           'employment_type', 
           'debt', 
           'monthly_income', 
           'purpose']
df.set_axis(columns, axis = 'columns', inplace = True)

Проверка переименования

In [12]:
df.columns

Index(['children', 'days_employed', 'age', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'employment_type',
       'debt', 'monthly_income', 'purpose'],
      dtype='object')

#### Рассмотрим и обработаем аномалии в столбцах <a class="anchor" id="subpoint2_1"></a>

Обработаем некорректную информацию в столбце с количеством детей. Количество детей является одним из запрошенных в задаче факторов влияния нафакт погашения кредита в срок. факт погашения кредита в срок.

Оценим объем строк с некореектными данными "-1" и "20"

In [13]:
df.groupby('children')['children'].count()

children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

Уберем значение "-1" - вероятнее всего, это некорректная выгрузка данных, предполагалось число "1"

In [14]:
df.loc[df['children'] == -1, 'children'] = df['children'].abs()

Оценим, в каких возрастных категориях у клиентов по 20 детей

In [15]:
print('до 40 лет у', df[(df['children'] == 20) & (df['age'] <= 40)]['children'].count(), 'человек')
print('старше 40 лет у', df[(df['children'] == 20) & (df['age'] > 40)]['children'].count(), 'человек')

до 40 лет у 36 человек
старше 40 лет у 40 человек


Вероятнее всего, 20 детей - это некорректная выгрузка данных, т.к. почти половина таких клиентов младше 40 лет, либо если предположить, что у клиентов дети приемные, смущает, что у всех клиентов их именно 20, ровно.  
Заменим 20 детей на 2

In [16]:
df.loc[df['children'] == 20, 'children'] = df['children'].replace(20, 2)

Проверим, что замены произведены

In [17]:
df.groupby('children')['children'].count()

children
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Обработаем столбцы с уровнем образования и семейным положением, уберем дубликаты с помощью приведения всех значений к нижнему регистру методом str.lower()

In [18]:
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()

Проверим, что замены произведены

In [19]:
print(df['education'].unique())
print(df['family_status'].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']


Теперь в столбцах по 5 уникальных значений, как в столбцах с соответствующими id.

Обработаем нулевые значения в столбце с возрастом клиента.

In [20]:
# оценим среднее и медиану
print('среднее', df[df['age'] != 0]['age'].mean())
print('медиана', df[df['age'] != 0]['age'].median())

среднее 43.497479462285284
медиана 43.0


In [21]:
# В задании нет цели оценить влияние возраста на факт возврата кредита в срок, 
# поэтому итоговая картина не будет искажена, если заменим нулевые значения на медиану
df.loc[df['age'] == 0, 'age'] = df['age'].replace(0, int(df[df['age'] != 0]['age'].median()))

Проверим, что замена произведена

In [22]:
df['age'].sort_values().unique()

array([19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
       70, 71, 72, 73, 74, 75])

Обработаем некорректное значение XNA в столбце с полом клиента.

Оценим, сколько значений в столбце с полом клиента имеют некорректное значение 'XNA'

In [23]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Всего одна запись. На общую картину, вероятно, не повлияет, если заменим ее, например на 'F'.  
Проверим, что замена произведена.

In [24]:
df.loc[df['gender'] == 'XNA', 'gender'] = df['gender'].replace('XNA', 'F')
df['gender'].value_counts()

F    14237
M     7288
Name: gender, dtype: int64

#### Вывод

Выявленные при первичном рассмотрении датасета аномалии устранены.

### Обработка пропусков <a class="anchor" id="subpoint2_2"></a>

Рассмотрим и обработаем строки с пропущенными значениями.

Определим количества пропущенных значений

In [25]:
df.isnull().sum()

children               0
days_employed       2174
age                    0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
employment_type        0
debt                   0
monthly_income      2174
purpose                0
dtype: int64

Удалять пропуски не стоит - потеряется примерно 10% датасета.
Одинаковое количество значений пропущено в столбцах с днями трудоустройства и месячным доходом. Возможно, пропуски в одних и тех же строках.

Посмотрим строки, в которых пропущены значения в столбце с днями трудоустройства 

In [26]:
days_employed_NaN = df[df['days_employed'].isnull()]

In [27]:
days_employed_NaN.head()

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,employment_type,debt,monthly_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Проверим, есть ли непустые значения в столбце с месячным доходом в тех строках, где пропущены значения в столбце с днями трудоустройства

In [28]:
days_employed_NaN['monthly_income'].unique()

array([nan])

Посмотрим, в каких категориях типа занятости 'employment_type' пропущены значения

In [29]:
days_employed_NaN['employment_type'].unique()

array(['пенсионер', 'госслужащий', 'компаньон', 'сотрудник',
       'предприниматель'], dtype=object)

Обработаем пустые и отрицательные значения в столбце со стажем в днях.

Исключив строки с пустыми значениями, посмотрим, сколько значений больше 0, и рассмотрим верх таблицы с положительными значениями, отсортированными по возрастанию:

In [30]:
print(df[df['days_employed'] >= 0]['days_employed'].dropna().count())
df[df['days_employed'] >= 0].dropna().sort_values(by='days_employed').head()

3445


,children,days_employed,age,education,education_id,family_status,family_status_id,gender,employment_type,debt,monthly_income,purpose
20444,0,328728.720605,72,среднее,1,вдовец / вдова,2,F,пенсионер,0,96519.339647,покупка жилья для семьи
9328,2,328734.923996,41,высшее,0,женат / замужем,0,M,пенсионер,0,126997.497760,операции со своей недвижимостью
17782,0,328771.341387,56,среднее,1,женат / замужем,0,F,пенсионер,0,68648.047062,операции с коммерческой недвижимостью
14783,0,328795.726728,62,высшее,0,женат / замужем,0,F,пенсионер,0,79940.196752,на покупку своего автомобиля
7229,1,328827.345667,32,среднее,1,гражданский брак,1,F,пенсионер,0,122162.965695,сыграть свадьбу


Всего лишь 16% значений в столбце с трудовым стажем имеют положительное значение. А минимальное положительное значение 328728.720605.

Рассмотрим таблицу со значениями, меньше минимального положительного

In [31]:
min_days_empl_above0 = df[df['days_employed'] >= 0]['days_employed'].dropna().min()
df[df['days_employed'] < min_days_empl_above0].sort_values(by='days_employed', ascending = False).head()

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,employment_type,debt,monthly_income,purpose
17437,1,-24.141633,31,среднее,1,женат / замужем,0,F,сотрудник,1,166952.415427,высшее образование
8336,0,-24.240695,32,высшее,0,не женат / не замужем,4,M,сотрудник,0,124115.373655,получение дополнительного образования
6157,2,-30.195337,47,среднее,1,гражданский брак,1,M,компаньон,0,231461.185606,свадьба
9683,0,-33.520665,43,среднее,1,не женат / не замужем,4,M,сотрудник,1,128555.897209,приобретение автомобиля
2127,1,-34.701045,31,высшее,0,женат / замужем,0,F,компаньон,0,90557.994311,получение образования


In [32]:
df[df['days_employed'] < min_days_empl_above0].sort_values(by='days_employed', ascending = False).tail()

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,employment_type,debt,monthly_income,purpose
16825,0,-16119.687737,64,среднее,1,женат / замужем,0,F,сотрудник,0,91527.685995,покупка жилой недвижимости
17838,0,-16264.699501,59,среднее,1,женат / замужем,0,F,сотрудник,0,51238.967133,на покупку автомобиля
7329,0,-16593.472817,60,высшее,0,женат / замужем,0,F,сотрудник,0,124697.846781,заняться высшим образованием
4299,0,-17615.563266,61,среднее,1,женат / замужем,0,F,компаньон,0,122560.741753,покупка жилья
16335,1,-18388.949901,61,среднее,1,женат / замужем,0,F,сотрудник,0,186178.934089,операции с недвижимостью


Положительные и отрицательные значения в столбце с трудовым стажем имеют разную размерность. Вероятно, отрицательные значения указаны в днях, как и предполагает датасет, а положительные значения указаны в часах. Переведем все значения к одной размерности - в годах и создадим новый столбец.

Переведем положительные значения из часов в дни 

In [33]:
df.loc[df['days_employed'] > 0, 'days_employed'] = df['days_employed'] / 24

Уберем отрицательные значения

In [34]:
df.loc[df['days_employed'] < 0, 'days_employed'] = df['days_employed'] *-1

Проверим результат

In [35]:
df.sort_values(by='days_employed').head()

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,employment_type,debt,monthly_income,purpose
17437,1,24.141633,31,среднее,1,женат / замужем,0,F,сотрудник,1,166952.415427,высшее образование
8336,0,24.240695,32,высшее,0,не женат / не замужем,4,M,сотрудник,0,124115.373655,получение дополнительного образования
6157,2,30.195337,47,среднее,1,гражданский брак,1,M,компаньон,0,231461.185606,свадьба
9683,0,33.520665,43,среднее,1,не женат / не замужем,4,M,сотрудник,1,128555.897209,приобретение автомобиля
2127,1,34.701045,31,высшее,0,женат / замужем,0,F,компаньон,0,90557.994311,получение образования


In [36]:
df.sort_values(by='days_employed').dropna().tail()

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,employment_type,debt,monthly_income,purpose
7664,1,16736.462226,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
10006,0,16738.158823,69,высшее,0,не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования
6954,0,16739.808353,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью
4299,0,17615.563266,61,среднее,1,женат / замужем,0,F,компаньон,0,122560.741753,покупка жилья
16335,1,18388.949901,61,среднее,1,женат / замужем,0,F,сотрудник,0,186178.934089,операции с недвижимостью


Заполним пропуски в стаже по медиане в зависимости от возраста

In [37]:
ages = df['age'].sort_values().unique()
try:
    for age in ages:
        median_days_employed = df[df['age'] == age]['days_employed'].dropna().median()
        df.loc[df['age'] == age, 'days_employed'] = df['days_employed'].fillna(median_days_employed)
except:
    print('НЕВЕРНЫЙ КОД!')

Проверим замену пустых значений в столбце со стажем в днях

In [38]:
df['days_employed'].isnull().sum()

0

Для наглядности добавим в таблицу столбец со стажем в годах

In [39]:
df['years_employed'] = df['days_employed'] / 365

In [40]:
df.head()

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,employment_type,debt,monthly_income,purpose,years_employed
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,23.116912
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,11.026860
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,15.406637
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,11.300677
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,38.843159


Обработаем пустые значения в столбце с месячным доходом

Заполним пропуски в столбце с месячным доходом по медиане в зависимости от типа занятости

In [41]:
employment_types = df['employment_type'].unique()
try:
    for variant in employment_types:
        median_monthly_income = df[df['employment_type'] == variant]['monthly_income'].median()
        df.loc[df['employment_type'] == variant, 'monthly_income'] = df['monthly_income'].fillna(median_monthly_income)
except:
    print('НЕВЕРНЫЙ КОД!')

Проверим замену пустых значений

In [42]:
df.isnull().sum()

children            0
days_employed       0
age                 0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
employment_type     0
debt                0
monthly_income      0
purpose             0
years_employed      0
dtype: int64

#### Вывод

Пустые значения в столбцах с днями трудоустройства и месячным доходом находятся в одних и тех же строках. Судя по значениям типа занятости 'employment_type', в данных строках должны быть непустые/ненулевые значения, т.к. категории относятся к трудоустроенному населению (не 'безработный', не 'студент').
Банку следует изучить природу их возникновения. Возможно, эти клиенты не предоставили справку 2-НДФЛ, содержащую сведения о трудоустройстве и доходах, либо произошла ошибка при выгрузке данных.
Также банку следует обратить внимание на порядок выгрузки данных в столбце со стажем в жнях. Вероятно, стаж, указанный в днях, выгружается как отрицательные значения. А положительные значения в столбце нереально велики для стажа в днях, возможно, положительные значения представляют собой стаж в часах.

### Замена типа данных <a class="anchor" id="subpoint2_3"></a>

Для облегчения зрительного восприятия датасета приведем значения в столбцах со стажем в днях и месячным доходом к целочисленным, а значения в новом столбце со стажем в годах округлим до двух знаков после запятой.

Так как нам необходимо получить целочисленный результат в столбцах со стажем в днях и месячным доходом, воспользуемся методом astype().

Приведем стаж к целочисленному значению

In [43]:
df['days_employed'] = df['days_employed'].astype('int64')

Месячный доход до шести знаков после запятой - неудобен для восприятия. Приведение значений к целочисленным не повлияет на результаты исследования.

Приведем месячный доход к целочисленному значению

In [44]:
df['monthly_income'] = df['monthly_income'].astype('int64')

Стаж в годах округлим до двух знаков после запятой, затем приведем к вещественному числу

In [45]:
df[['years_employed']] = df[['years_employed']].applymap("{0:.2f}".format)
df['years_employed'] = pd.to_numeric(df['years_employed'])

Проверим внесеннные изменения

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
age                 21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
employment_type     21525 non-null object
debt                21525 non-null int64
monthly_income      21525 non-null int64
purpose             21525 non-null object
years_employed      21525 non-null float64
dtypes: float64(1), int64(7), object(5)
memory usage: 2.1+ MB


Посмотрим обновленный датасет

In [47]:
df.sort_values(by='years_employed').head()

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,employment_type,debt,monthly_income,purpose,years_employed
17437,1,24,31,среднее,1,женат / замужем,0,F,сотрудник,1,166952,высшее образование,0.07
8336,0,24,32,высшее,0,не женат / не замужем,4,M,сотрудник,0,124115,получение дополнительного образования,0.07
6157,2,30,47,среднее,1,гражданский брак,1,M,компаньон,0,231461,свадьба,0.08
9683,0,33,43,среднее,1,не женат / не замужем,4,M,сотрудник,1,128555,приобретение автомобиля,0.09
2127,1,34,31,высшее,0,женат / замужем,0,F,компаньон,0,90557,получение образования,0.10


#### Вывод

После замены типа данных в столбцах со стажем в днях и месячным доходом анализируемая таблица стала удобней для восприятия и обработки.

### Обработка дубликатов <a class="anchor" id="subpoint2_4"></a>

Проверим, содержит ли датасет дубликаты, и обработаем их при наличии.

Найдем число дубликатов

In [48]:
df.duplicated().sum()

71

Небольшое количество дублей имеется. Возможно, произошла некая ошибка при выгрузке данных, либо, например, клиенты, сделавшие рефинансирование кредита под меньший процент или другой срок в этом же банке, снова попали в статистику. Банку следует изучить вопрос появления дубликатов в статистике.
Так как число дубликатов относительно общего числа строк датасета невелико, удалим дубликаты методом drop_duplicates().

Удалим дубликаты и сбросим индексы

In [49]:
df = df.drop_duplicates().reset_index(drop=True)

Проверим, что дубликаты удалены

In [50]:
df.duplicated().sum()

0

#### Вывод

Датасет очищен от дубликатов. Банку следует изучить природу их возникновения в статистике. 

### Лемматизация <a class="anchor" id="subpoint2_5"></a>

В столбце с целями кредита 'purpose' одинаковые цели описаны разными словосочетаниями. Столбец содержит 38 уникальных значений, многие из которых повторяют друг друга по смыслу. Проведем лемматизацию, категоризируем и сократим список уникальных значений для целей кредита.

Посчитаем количество уникальных значений

In [51]:
len(df['purpose'].unique())

38

Посмотрим список уникальных значений

In [52]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [53]:
# добавим пробел в конце значений целей кредита, чтобы при объединении в 1 строку они не сливались со следующим значением
purposes = df['purpose'].unique()
for purpose in purposes:
    purpose2 = purpose + ' '
    df.loc[df['purpose'] == purpose, 'purpose'] = df['purpose'].replace(purpose, purpose2)

# объединим цели в 1 строку для лемматизации    
purposes2 = df['purpose'].unique()
text = ''
for i in purposes2:
    text += i
    
# лемматизируем цели кредита и проведем подсчет числа упоминаний лемматизированных слов
purposes_lem = m.lemmatize(text)
print(Counter(purposes_lem))

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, ' \n': 1})


На основании лемматизации выделим основные цели кредита.

In [54]:
credit_purposes = ['недвижимость', 'автомобиль', 'образование', 'жилье', 'свадьба']

Добавим в датасет столбец с лемматизацией цели кредита

In [55]:
def lemma(text):
    try:
        lemtext = ''
        lemtext += text    
        return m.lemmatize(lemtext)
    except:
        print('НЕВЕРНЫЙ КОД!')
df['purposes_lem'] = df['purpose'].apply(lemma)

Добавим в датасет столбец с выделенными основными целями кредита после лемматизации

In [56]:
def lemma_repl(text2):
    try:
        for element in credit_purposes:
            if element in text2:
                return element
    except:
        print('НЕВЕРНЫЙ КОД!')
df['credit_purpose'] = df['purposes_lem'].apply(lemma_repl)

В столбце с основными целями кредита после лемматизации - 'credit_purpose' - заменим значение 'жилье' на 'недвижимость'

In [57]:
df.loc[df['credit_purpose'] == 'жилье', 'credit_purpose'] = df['credit_purpose'].replace('жилье', 'недвижимость')

Посмотрим результат

In [58]:
display(df.head())

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,employment_type,debt,monthly_income,purpose,years_employed,purposes_lem,credit_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23.12,"[покупка, , жилье, \n]",недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11.03,"[приобретение, , автомобиль, \n]",автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15.41,"[покупка, , жилье, \n]",недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11.30,"[дополнительный, , образование, \n]",образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,38.84,"[сыграть, , свадьба, \n]",свадьба


Проверим, что заполнился весь датасет

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 15 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
age                 21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
employment_type     21454 non-null object
debt                21454 non-null int64
monthly_income      21454 non-null int64
purpose             21454 non-null object
years_employed      21454 non-null float64
purposes_lem        21454 non-null object
credit_purpose      21454 non-null object
dtypes: float64(1), int64(7), object(7)
memory usage: 2.5+ MB


Проверим, что в целях кредита осталось только 4 уникальных значения

In [60]:
df['credit_purpose'].unique()

array(['недвижимость', 'автомобиль', 'образование', 'свадьба'],
      dtype=object)

Удалим старый стобец с целями кредита и столбец с лемматизацией целей

In [61]:
del df['purpose']
del df['purposes_lem']

Посмотрим обновленный датасет

In [62]:
display(df.head())

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,employment_type,debt,monthly_income,years_employed,credit_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,23.12,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,11.03,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,15.41,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,11.30,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,38.84,свадьба


#### Вывод

С помощью лемматизации удалось 38 значений в столбце с целями кредита привести всего к 4 уникальным значениям целей кредита. Это позволит сформулировать выводы и дать ответ на поставленный в задаче вопрос о наличии влияния целей кредита на наличие у клиента задолженности по возврату кредита в срок.

### Категоризация данных <a class="anchor" id="subpoint2_6"></a>

Проведем категоризацию клиентов банка по уровню их дохода. Это впоследствии позволит дать ответ на поставленную задачу, существует ли зависимость между уровнем дохода и возвратом кредита в срок.  

Для выделения категорий оценим статистические данные по месячному доходу клиентов банка разделим месячный доход на 1000 для большей наглядности

In [63]:
monthly_income_kilo = df['monthly_income']/1000
monthly_income_kilo.describe()

count    21454.000000
mean       165.319572
std         98.187303
min         20.667000
25%        107.623000
50%        142.594000
75%        195.820250
max       2265.604000
Name: monthly_income, dtype: float64

Так как разница между min и max колоссальная, для верного выделения категорий посмотрим более детально, как распределяются доходы среди клиентов: в каких уровнях доходов максимальное количество клиентов

In [64]:
range_check = [50, 100, 150, 200, 250, 300, 500, 700, 1000, 1500, 2000, 2500]
def range_income_group(income):
    try:
        for element in range_check:
            if income < element:
                return element
    except:
        print('НЕВЕРНЫЙ КОД!')
monthly_income_kilo['range'] = monthly_income_kilo.apply(range_income_group)

In [65]:
monthly_income_kilo['range'].value_counts().reset_index().sort_values(by='index')

,index,monthly_income
6,50,372
2,100,4091
0,150,7160
1,200,4764
3,250,2254
4,300,1330
5,500,1261
7,700,161
8,1000,36
9,1500,18


Разделим на категории с помощью функции

In [66]:
def income_group(income):
    try:
        if income < 100000:
            return 'низкий'
        if income < 250000:
            return 'средний'
        if income < 500000:
            return 'высокий'
        return 'сверхвысокий'
    except:
        print('НЕВЕРНЫЙ КОД!')

Добавим столбец с категориями в исходный датасет

In [67]:
df['income_group'] = df['monthly_income'].apply(income_group)

Посмотрим, как клиенты распределились по группам в зависимости от дохода

In [68]:
df['income_group'].value_counts()

средний         14178
низкий           4463
высокий          2591
сверхвысокий      222
Name: income_group, dtype: int64

Посмотрим обновленный датасет

In [69]:
df.head()

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,employment_type,debt,monthly_income,years_employed,credit_purpose,income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,23.12,недвижимость,высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,11.03,автомобиль,средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,15.41,недвижимость,средний
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,11.30,образование,высокий
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,38.84,свадьба,средний


В связи с тем, что доля клиентов банка с 3,4,5 детьми в анализируемой выборке очень мала, проведем категоризацию клиентов и выделим следующие категории: "0 детей" / "1 ребенок" / "2 и более детей". Это позволит дать более достоверный ответ на поставленную задачу, существует ли зависимость между количеством детей клиента и возвратом кредита в срок.

In [70]:
def children_grouped(children):
    try:
        if children < 1:
            return '0 детей'
        if children < 2:
            return '1 ребенок'
        return '2 и более детей'
    except:
        print('НЕВЕРНЫЙ КОД!')

Добавим столбец с новыми категориями по количеству детей в исходный датасет

In [71]:
df['children_grouped'] = df['children'].apply(children_grouped)

Посмотрим новое распределение клиентов по группам в зависимости от количества детей

In [72]:
df['children_grouped'].value_counts()

0 детей            14091
1 ребенок           4855
2 и более детей     2508
Name: children_grouped, dtype: int64

Посмотрим обновленный датасет

In [73]:
df.head()

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,employment_type,debt,monthly_income,years_employed,credit_purpose,income_group,children_grouped
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,23.12,недвижимость,высокий,1 ребенок
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,11.03,автомобиль,средний,1 ребенок
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,15.41,недвижимость,средний,0 детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,11.30,образование,высокий,2 и более детей
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,38.84,свадьба,средний,0 детей


#### Вывод

Проведя категоризацию, выделили в датасете 4 уровня доходов клиентов. Это позволит сформулировать выводы и дать ответ на поставленный в задаче вопрос о наличии влияния уровня дохода клиента на наличие у него задолженности по возврату кредита в срок.

### 3. Расчеты для ответа на поставленные в задаче вопросы <a class="anchor" id="point3"></a>

Вычислим средний по датасету процент клиентов, имеющих задолженность по возврату кредита в срок

In [74]:
debt = df.loc[df['debt'] == 1]['debt'].count()
mean_debt_percentage = debt / len(df)
print('Средний по датасету процент клиентов с долгом {:.2%}'.format(mean_debt_percentage))

Средний по датасету процент клиентов с долгом 8.12%


Оценим, как влияет количество детей у клиента на наличие у него задолженности по возврату кредита в срок

Составим сводную таблицу в разрезе количества детей у клиента и наличия у него задолженности по возврату кредита в срок

In [75]:
children_pivot = df.pivot_table(index=['children_grouped'], columns='debt', values='credit_purpose', aggfunc='count')

In [76]:
children_pivot

debt,0,1
children_grouped,,
0 детей,13028,1063
1 ребенок,4410,445
2 и более детей,2275,233


Вычислим процент клиентов, имеющих задолженность

In [77]:
children_pivot['debt_percentage'] = children_pivot[1] / (children_pivot[0] + children_pivot[1])

Создадим функцию, которая будет сравнивать рассчитанные в зависимости от количества детей проценты клиентов, имеющих задолженность, со средним процентом клиентов с долгом по всему датасету.

In [78]:
def comparison_with_mean(percentage):
    try:
        if percentage > mean_debt_percentage:
            return 'выше среднего'
        return '_'
    except:
        print('НЕВЕРНЫЙ КОД!')

Добавим в таблицу столбец со сравнением

In [79]:
children_pivot['debt_percentage_compared'] = children_pivot['debt_percentage'].apply(comparison_with_mean)

Для наглядности добавим долю группы клиентов в общем количестве клиентов

In [80]:
children_pivot['children_qty_percentage'] = df['children_grouped'].value_counts(normalize = True)

Чтобы проценты выводились в процентном формате, применим форматирование

In [81]:
children_pivot[['debt_percentage']] = children_pivot[['debt_percentage']].applymap("{0:.2%}".format)
children_pivot[['children_qty_percentage']] = children_pivot[['children_qty_percentage']].applymap("{0:.2%}".format)

Оценим, как влияет семейное положение клиента на наличие у него задолженности по возврату кредита в срок.

Составим сводную таблицу в разрезе семейного положения клиента и наличия у него задолженности по возврату кредита в срок

In [82]:
family_status_pivot = df.pivot_table(index=['family_status'], columns='debt', values='credit_purpose', aggfunc='count')

In [83]:
family_status_pivot

debt,0,1
family_status,,
в разводе,1110,85
вдовец / вдова,896,63
гражданский брак,3763,388
женат / замужем,11408,931
не женат / не замужем,2536,274


Вычислим процент клиентов, имеющих задолженность

In [84]:
family_status_pivot['debt_percentage'] = family_status_pivot[1] / (family_status_pivot[0] + family_status_pivot[1])

Добавим в таблицу столбец со сравнением со средним процентом должников

In [85]:
family_status_pivot['debt_percentage_compared'] = family_status_pivot['debt_percentage'].apply(comparison_with_mean)

Для наглядности добавим долю группы клиентов в общем количестве клиентов

In [86]:
family_status_pivot['family_status_percentage'] = df['family_status'].value_counts(normalize = True)

Чтобы проценты выводились в процентном формате, применим форматирование

In [87]:
family_status_pivot[['debt_percentage']] = family_status_pivot[['debt_percentage']].applymap("{0:.2%}".format)
family_status_pivot[['family_status_percentage']] = family_status_pivot[['family_status_percentage']].applymap("{0:.2%}".format)

Оценим, как влияет уровень дохода клиента на наличие у него задолженности по возврату кредита в срок.

Составим сводную таблицу в разрезе уровня дохода клиента и наличия у него задолженности по возврату кредита в срок

In [88]:
income_group_pivot = df.pivot_table(index=['income_group'], columns='debt', values='credit_purpose', aggfunc='count')

In [89]:
income_group_pivot

debt,0,1
income_group,,
высокий,2411,180
низкий,4109,354
сверхвысокий,208,14
средний,12985,1193


Вычислим процент клиентов, имеющих задолженность

In [90]:
income_group_pivot['debt_percentage'] = income_group_pivot[1] / (income_group_pivot[0] + income_group_pivot[1])

Добавим в таблицу столбец со сравнением со средним процентом должников

In [91]:
income_group_pivot['debt_percentage_compared'] = income_group_pivot['debt_percentage'].apply(comparison_with_mean)

Для наглядности добавим долю группы клиентов в общем количестве клиентов

In [92]:
income_group_pivot['income_group_percentage'] = df['income_group'].value_counts(normalize = True)

Чтобы проценты выводились в процентном формате, применим форматирование

In [93]:
income_group_pivot[['debt_percentage']] = income_group_pivot[['debt_percentage']].applymap("{0:.2%}".format)
income_group_pivot[['income_group_percentage']] = income_group_pivot[['income_group_percentage']].applymap("{0:.2%}".format)

Оценим, как влияет цель кредита на наличие у клиента задолженности по возврату кредита в срок.

Составим сводную таблицу в разрезе целей кредите и наличия у клиента задолженности по возврату кредита в срок

In [94]:
credit_purpose_pivot = df.pivot_table(index=['credit_purpose'], columns='debt', values='income_group', aggfunc='count')

In [95]:
credit_purpose_pivot

debt,0,1
credit_purpose,,
автомобиль,3903,403
недвижимость,10029,782
образование,3643,370
свадьба,2138,186


Вычислим процент клиентов, имеющих задолженность

In [96]:
credit_purpose_pivot['debt_percentage'] = credit_purpose_pivot[1] / (credit_purpose_pivot[0] + credit_purpose_pivot[1])

Добавим в таблицу столбец со сравнением со средним процентом должников

In [97]:
credit_purpose_pivot['debt_percentage_compared'] = credit_purpose_pivot['debt_percentage'].apply(comparison_with_mean)

Для наглядности добавим долю группы клиентов в общем количестве клиентов

In [98]:
credit_purpose_pivot['credit_purpose_percentage'] = df['credit_purpose'].value_counts(normalize = True)

Чтобы проценты выводились в процентном формате, применим форматирование

In [99]:
credit_purpose_pivot[['debt_percentage']] = credit_purpose_pivot[['debt_percentage']].applymap("{0:.2%}".format)
credit_purpose_pivot[['credit_purpose_percentage']] = credit_purpose_pivot[['credit_purpose_percentage']].applymap("{0:.2%}".format)

Попробуем оценить, при совокупности каких признаков процент клиентов с долгом максимален. На эти категории банку нужно обращать внимание в первую очередь при оценке риска невозврата кредита клиентом в срок.

Составим сводную таблицу в разрезе всех определенных задачей факторов и наличия у клиента задолженности по возврату кредита в срок

In [100]:
df_pivot = df.pivot_table(index=['children', 'family_status', 'income_group', 'credit_purpose', ], columns='debt', values='employment_type', aggfunc='count')

Вычислим процент клиентов, имеющих задолженность

In [101]:
df_pivot['debt_percentage'] = df_pivot[1] / (df_pivot[0] + df_pivot[1])

Добавим в таблицу столбец со сравнением со средним процентом должников

In [102]:
df_pivot['debt_percentage_compared'] = df_pivot['debt_percentage'].apply(comparison_with_mean)

Создадим из сводной таблицы новый датафрейм для удобства дальнейшего анализа

In [103]:
df_pivot = df_pivot.dropna().reset_index()
new_df = pd.DataFrame(df_pivot.to_records())
new_df['debt_percentage'] = new_df['debt_percentage'] * 100

Добавим столбец с долей каждой полученной категории (в новой строке) клиентов относительно общего количества записей в исходном датасете, т.к. имеет смысл рассматривать только относительно репрезентативные выборки

In [104]:
new_df['representative_stat'] = (new_df['0'] + new_df['1']) / len(df) * 100

Отфильтруем новую таблицу по доле выборки от 1.5% относительно изначального датасета и по проценту должников, превышающему средний процент по изначальному датасету

In [105]:
most_debt = new_df.loc[(new_df['representative_stat'] > 1.5) & (new_df['debt_percentage_compared'] == 'выше среднего')]

Расчеты завершены, вывод итоговых сводным таблиц см. в Ответах на поставленные в задаче вопросы.

### 4. Ответы на поставленные в задаче вопросы <a class="anchor" id="point4"></a>

- **Есть ли зависимость между наличием детей и возвратом кредита в срок?** <a class="anchor" id="question1"></a>

In [106]:
children_pivot

debt,0,1,debt_percentage,debt_percentage_compared,children_qty_percentage
children_grouped,,,,,
0 детей,13028,1063,7.54%,_,65.68%
1 ребенок,4410,445,9.17%,выше среднего,22.63%
2 и более детей,2275,233,9.29%,выше среднего,11.69%


#### Вывод

Клиенты, у которых нет детей, реже других имеют задолженность по возврату кредита в срок, а так же реже, чем в среднем по датасету. Такие Клиенты составляют 65,7% от датасета, можно предположить, что выборка по ним более информативна, чем по другим. Доля Клиентов с 3, 4 и 5 детьми в датасете совсем мала, чтобы опираться в работе на их статистику, поэтому была проведена категоризация на группы "0 детей" / "1 ребенок" / "2 и более детей".  
В целом можно сделать вывод, что наличие детей у клиента увеличивает риск невозрата им кредита в срок.

- **Есть ли зависимость между семейным положением и возвратом кредита в срок?** <a class="anchor" id="question2"></a>

In [107]:
family_status_pivot

debt,0,1,debt_percentage,debt_percentage_compared,family_status_percentage
family_status,,,,,
в разводе,1110,85,7.11%,_,5.57%
вдовец / вдова,896,63,6.57%,_,4.47%
гражданский брак,3763,388,9.35%,выше среднего,19.35%
женат / замужем,11408,931,7.55%,_,57.51%
не женат / не замужем,2536,274,9.75%,выше среднего,13.10%


#### Вывод

Более высокий риск невозврата кредита в срок у клиентов, которые состоят в 'гражданском браке' либо холосты. Процент таких клиентов, имеющих задолженность, выше, чем в среднем по датасету. Самый низкий процент клиентов с задолженностью в категориях 'вдовец / вдова' и 'в разводе', но их доля в датасете невелика, есть риск, что статистика непоказательна. Чуть ниже среднего по датасету риск невозврата кредита в срок среди клиентов, состоящих в браке.

- **Есть ли зависимость между уровнем дохода и возвратом кредита в срок?** <a class="anchor" id="question3"></a>

In [108]:
income_group_pivot

debt,0,1,debt_percentage,debt_percentage_compared,income_group_percentage
income_group,,,,,
высокий,2411,180,6.95%,_,12.08%
низкий,4109,354,7.93%,_,20.80%
сверхвысокий,208,14,6.31%,_,1.03%
средний,12985,1193,8.41%,выше среднего,66.09%


#### Вывод

Наиболее высокий процент клиентов, имеющих задолженность, - клиенты со средним доходом. Реже других имеют задолженность по возврату кредита в срок клиенты со сверхвысоким доходом. Также ниже среднего процент клиентов, имеющих задолженность, среди клиентов с высоким доходом. Чуть ниже среднего по датасету риск невозврата кредита в срок среди клиентов, имеющих низкий доход.

- **Как разные цели кредита влияют на его возврат в срок?** <a class="anchor" id="question4"></a>

In [109]:
credit_purpose_pivot

debt,0,1,debt_percentage,debt_percentage_compared,credit_purpose_percentage
credit_purpose,,,,,
автомобиль,3903,403,9.36%,выше среднего,20.07%
недвижимость,10029,782,7.23%,_,50.39%
образование,3643,370,9.22%,выше среднего,18.71%
свадьба,2138,186,8.00%,_,10.83%


#### Вывод

Наиболее высокий процент клиентов, имеющих задолженность, - клиенты, которые взяли кредит на автомобиль. Также выше среднего процент клиентов с задолженностью - клиенты, взявшие кредит на образование. Реже всех имеют задолженность по возврату кредита в срок клиенты, взявшие кредит на недвижимость. Также чуть ниже среднего процент клиентов с задолженностью среди клиентов, которые взяли кредит на свадьбу.

### 5. Общий вывод <a class="anchor" id="point5"></a>

Проведенный анализ показал, что все рассмотренные параметры - семейное положение, количество детей, цели кредита и уровень дохода клиента - имеют влияние на возврат кредита клиентом в срок. Вероятность невозврата кредита клиентом в срок меньше всего в категории граждан со сверхвысоким доходом. Вышего всего риск невозврата кредита в срок у холостых клиентов, у клиентов с двумя детьми, а также у клиентов, берущих кредит на автомобиль.
Наличие детей у клиента определенно увеличивает риск невозрата им кредита в срок. Клиенты, состоящие в официальном браке, определенно надежнее холостых клиентов, а также клиентов, состоящих в "гражданском браке". Также чаще других имеют задолженность клиенты со средним доходом.

При оценке риска невозврата кредита клиентом в срок Банку следует обращать особое внимание на клиентов при наличии у него совокупности признаков, указанных в таблице ниже. Процент таких клиентов с долгом максимален.

In [110]:
most_debt.sort_values(by = 'debt_percentage', ascending = False)

,index,children,family_status,income_group,credit_purpose,0,1,debt_percentage,debt_percentage_compared,representative_stat
51,51,0,не женат / не замужем,средний,автомобиль,312.0,50.0,13.812155,выше среднего,1.687331
83,83,1,женат / замужем,средний,автомобиль,408.0,48.0,10.526316,выше среднего,2.125478
85,85,1,женат / замужем,средний,образование,366.0,42.0,10.294118,выше среднего,1.901743
75,75,1,гражданский брак,средний,свадьба,334.0,33.0,8.991826,выше среднего,1.710637
120,120,2,женат / замужем,средний,недвижимость,515.0,49.0,8.687943,выше среднего,2.628880
28,28,0,гражданский брак,средний,недвижимость,409.0,38.0,8.501119,выше среднего,2.083528
30,30,0,гражданский брак,средний,свадьба,931.0,84.0,8.275862,выше среднего,4.731052
